# Counter Strike Coaching Strategies - Analysis

### Import class and libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.patches as mpatches
import matplotlib.image as mpimg
import numpy as np
import seaborn as sns
from process_game_state import ProcessGameState


### Create game state instance

In [ ]:
game_state = ProcessGameState('data/game_state_frame_data.parquet')
game_state.data.head()


## Question 1: Is entering via the light blue boundary a common strategy used by Team 2 on T side?

In [ ]:
# Define the boundaries for the light blue boundary
xy_bounds = [(-1735, 250), (-2024, 398), (-2806, 742),
             (-2472, 1233), (-1565, 580)]
closed_boundary_coords = np.concatenate((xy_bounds, [xy_bounds[0]]))
z_bounds = (285, 421)

# Get the min and max of the x and y coordinates for visualization
x_coords = [coord[0] for coord in xy_bounds]
y_coords = [coord[1] for coord in xy_bounds]

x_min = min(x_coords)
x_max = max(x_coords)
y_min = min(y_coords)
y_max = max(y_coords)


### Calculate the number of times Team 2 enters via the light blue boundary using a basic boundary detection algorithm

In [ ]:
# Get the number of times the player is in bounds for each round
team2_T_in_bounds = game_state.check_boundaries(
    xy_bounds, z_bounds, "Team2", "T")
print(team2_T_in_bounds)

# Get the number of rounds the boundary is entered
team2_T_entered = [True if num > 0 else False for num in team2_T_in_bounds]
team2_T_entered_count = {True: 0, False: 0}
for round in team2_T_entered:
    team2_T_entered_count[round] += 1

print("The number of rounds that Team 2 enters the boundary on T side is " +
      str(team2_T_entered_count[True]) + " out of " + str(len(team2_T_in_bounds)) + " rounds.")


### Calculate the number of times Team 2 enters via the light blue boundary using a more advanced boundary detection algorithm (Matplotlib Path class)

In [ ]:
# Get the number of times the player is in bounds for each round
team2_T_in_bounds = game_state.check_boundaries_with_matplotlib(
    closed_boundary_coords, z_bounds, "Team2", "T")
print(team2_T_in_bounds)

# Get the number of rounds the boundary is entered
team2_T_entered = [True if num > 0 else False for num in team2_T_in_bounds]
team2_T_entered_count = {True: 0, False: 0}
for round in team2_T_entered:
    team2_T_entered_count[round] += 1

print("The number of rounds that Team 2 enters the boundary on T side is " +
      str(team2_T_entered_count[True]) + " out of " + str(len(team2_T_in_bounds)) + " rounds.")


### Visualize the boundary detection algorithms

#### Basic boundary detection algorithm

In [ ]:
fig, ax = plt.subplots()
rect = mpatches.Rectangle((x_min, y_min), x_max - x_min,
                          y_max - y_min, fill=False, edgecolor='blue')
ax.add_patch(rect)
ax.set_xlim(x_min - 100, x_max + 100)  # Set the x-axis limits
ax.set_ylim(y_min - 100, y_max + 100)  # Set the y-axis limits
plt.gca().set_aspect('equal', adjustable='box')  # Set aspect ratio to equal
plt.show()


In [ ]:
boundary_path = mpath.Path(closed_boundary_coords)

fig, ax = plt.subplots()
patch = mpatches.PathPatch(boundary_path, facecolor='none', edgecolor='blue')
ax.add_patch(patch)
ax.set_xlim(x_min - 100, x_max + 100)  # Set the x-axis limits
ax.set_ylim(y_min - 100, y_max + 100)  # Set the y-axis limits
plt.gca().set_aspect('equal', adjustable='box')  # Set aspect ratio to equal
plt.show()


## Question 2: What is the average timer that Team 2 on T side enters Bombsite B with at least 2 rifles or SMGs?

### Extract weapon classes for each row

In [ ]:
game_state.extract_weapon_classes()
game_state.data.head()


In [ ]:
team2_T_data = game_state.data[(game_state.data['team'] == 'Team2') & (
    game_state.data['side'] == 'T') & (game_state.data['area_name'] == 'BombsiteB')]

team2_T_data = team2_T_data[team2_T_data['weapon_classes'].apply(
    lambda x: 'Rifle' in x or 'SMG' in x)]

# Create a list of the earliest timer in each round
earliest_timers = []
for round in team2_T_data['round_num'].unique():
    curr_round = team2_T_data[team2_T_data['round_num'] == round]
    earliest_timers.append(curr_round['clock_time'].max())

# Find the mean of the datetime objects in the list
mean_time = pd.to_datetime(earliest_timers, format="%M:%S").mean()

# Convert the mean time to a string
mean_time = mean_time.strftime("%M:%S")

print("The mean time that Team 2 enters the bombsite on T side with rifles or SMGs is " + mean_time + ".")


## Question 3: Where should we suspect Team 2 on CT side to be waiting inside Bombsite B?

In [ ]:
team2_CT_data = game_state.data[(game_state.data['team'] == 'Team2') & (
    game_state.data['side'] == 'CT') & (game_state.data['area_name'] == 'BombsiteB') & (game_state.data['bomb_planted'] == False)]

# Create a heatmap of the positions of the CT players overlaid with the image at map/de_overpass_radar.jpeg
img = mpimg.imread('map/de_overpass_radar.jpeg')
plt.figure(figsize=(20, 20))
fig, ax = plt.subplots()
ax.imshow(img, extent=[-5100, 0, -3750, 2000])
sns.histplot(x=team2_CT_data['x'], y=team2_CT_data['y'], cmap='Reds', bins=5, pthresh=0.05, cbar=True, ax=ax)
ax.set_xlim(-2806, 0)  # Set the x-axis limits
ax.set_ylim(-1233, 1233)  # Set the y-axis limits
plt.gca().set_aspect('equal', adjustable='box')  # Set aspect ratio to equal
plt.show()

